In [1]:
import pandas as pd
import numpy as np
import random

#split dataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

#AUCROC,AUCPR,precision,recall,f1-score
from sklearn.metrics import roc_curve,auc,average_precision_score,precision_score,recall_score,f1_score

#gridsearch/randomsearch
from itertools import product
from tqdm import tqdm

#visualize results
import matplotlib.pyplot as plt
import time

In [2]:
import torch
import torchvision
from torch import nn
from torch.autograd import Variable
from torch.utils.data import Subset,DataLoader,TensorDataset
from torchvision import datasets,transforms
import torch.nn.functional as F

In [3]:
# import warnings
# warnings.filterwarnings("ignore")

if torch.cuda.is_available():
  is_cuda=True
  print('GPU is on')
else:
  is_cuda=False
  print('GPU is off')

GPU is off


In [4]:
def set_seed(seed):
  np.random.seed(seed)
  torch.manual_seed(seed)
  random.seed(seed)

  torch.backends.cudnn.deterministic = True
  torch.backends.cudnn.benchmark = False

In [5]:
def data_prepare(batch_size,contam_ratio,hybrid,seed):
    set_seed(seed)
    data = pd.read_csv("E:/Research_Anomaly Detection/Taiwanese Bankruptcy Prediction (TBP)/data/TBP.csv")
    #change 0,1 label to 1,-1
    data.loc[data['Flag'] == 1,'Flag'] = -1
    data.loc[data['Flag'] == 0,'Flag'] = 1

    X = np.array(data.drop(['Flag'], axis = 1))
    #X = np.array(data.iloc[:,1:20])
    y = np.array(data['Flag'].values)
    
    #reverse
    scaler = MinMaxScaler().fit(X)
    X = scaler.inverse_transform(X)

    #split the data to training, validation and testing data (50%,20%,30%)
    X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.3,shuffle = False)
    X_train,X_val,y_train,y_val = train_test_split(X_train,y_train,test_size = 2/7,shuffle = False)

#     X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.3,shuffle = True,stratify = y)
#     X_train,X_val,y_train,y_val = train_test_split(X_train,y_train,test_size = 1/7,shuffle = True,stratify = y_train)

    #the known positive samples before contaminating
    known_pos_entire = sum(y_train == -1)
    #Minmax
    scaler=MinMaxScaler().fit(X_train)

    X_train = scaler.transform(X_train)
    X_val = scaler.transform(X_val)
    X_test = scaler.transform(X_test)

    index_contam = np.arange(len(y_train))[y_train == -1]
    index_contam = np.random.choice(index_contam,int(contam_ratio*len(index_contam)),replace = False)

    y_train[index_contam] = 1
    #the known positive samples after contaminating
    known_pos_sub = sum(y_train == -1)

    print(f'The left true(known) positive samples in the training set:{known_pos_sub}/{known_pos_entire}\n')
    #normal training tensor and dataloader
    #none-hybrid model only use "normal" samples in the training phase, which could be contaminated
    if not hybrid:
        index_subset = np.arange(len(y_train))[y_train == 1]
    else:
        index_subset = np.arange(len(y_train))
    

    #transform numpy to pytorch tensor
    train_tensor = TensorDataset(torch.from_numpy(X_train).float(),torch.tensor(y_train))
    train_tensor = Subset(train_tensor,index_subset)
    #fitting by batches (using dataloader), there exists randomness when shuffle=True***
    train_loader=DataLoader(train_tensor,batch_size = batch_size,shuffle = False,drop_last = True)
    
    #validation set
    X_val_tensor = torch.from_numpy(X_val).float()
    #testing set
    X_test_tensor=torch.from_numpy(X_test).float()
    
    return train_loader,X_val_tensor,y_val,X_test_tensor,y_test

In [6]:
class generator(nn.Module):
  def __init__(self,input_size,act_fun):
    super(generator,self).__init__()

    self.encoder_1=nn.Sequential(
      nn.Linear(input_size,input_size//4),
      act_fun,
      )
    
    self.decoder_1=nn.Sequential(
      nn.Linear(input_size//4,input_size),
      )

    self.encoder_2=nn.Sequential(
      nn.Linear(input_size,input_size//4),
      act_fun,
      )

  def forward(self,input):
    z=self.encoder_1(input)
    X_hat=self.decoder_1(z)
    z_hat=self.encoder_2(X_hat)

    return z,X_hat,z_hat

In [7]:
class discriminator(nn.Module):
  def __init__(self,input_size,act_fun):
    super(discriminator,self).__init__()

    self.encoder=nn.Sequential(
        nn.Linear(input_size,input_size//4),
        act_fun,
        )

    self.classifier=nn.Sequential(
        nn.Linear(input_size//4,1),
        nn.Sigmoid()
        )
    
  def forward(self,input):
    latent_vector=self.encoder(input)
    output=self.classifier(latent_vector)

    return latent_vector,output

In [8]:
def fit(dataloader,net_generator,net_discriminator,eta,epochs,batch_size,print_loss):
  L1_criterion = nn.L1Loss(reduction='none')
  L2_criterion = nn.MSELoss(reduction='none')
  BCE_criterion = nn.BCELoss(reduction='mean')

  if is_cuda:
    L1_criterion.cuda()
    L2_criterion.cuda()
    BCE_criterion.cuda()

  loss_D_list = []
  loss_G_list = []

  for epoch in range(epochs):
    for i,data in enumerate(dataloader):
      #y_aclabel means the acquired label information (which may be contaminated)
      X,y_aclabel = data
      y_real = torch.FloatTensor(batch_size).fill_(0)#real label=0,size=batch_size
      y_fake = torch.FloatTensor(batch_size).fill_(1)#fake label=1,size=batch_size

      if is_cuda:
        X = X.cuda()
        y_aclabel = y_aclabel.cuda()
        y_real = y_real.cuda()
        y_fake = y_fake.cuda()

      X = Variable(X)
      y_aclabel = Variable(y_aclabel)
      y_real = Variable(y_real)
      y_fake = Variable(y_fake)

      #zero grad for discriminator
      net_discriminator.zero_grad()
      #training the discriminator with real sample
      _,output = net_discriminator(X)
      loss_D_real = BCE_criterion(output.view(-1),y_real)

      #training the discriminator with fake sample
      _,X_hat,_ = net_generator(X)
      _,output = net_discriminator(X_hat)

      loss_D_fake = BCE_criterion(output.view(-1),y_fake)

      #entire loss in discriminator
      loss_D = (loss_D_real+loss_D_fake)/2
      
      loss_D.backward()
      optimizer_D.step()

      #training the generator based on the result from the discriminator
      net_generator.zero_grad()

      z,X_hat,z_hat = net_generator(X)

      #latent loss
      feature_real,_ = net_discriminator(X)
      feature_fake,_ = net_discriminator(X_hat)

      loss_G_latent = torch.mean(L2_criterion(feature_fake,feature_real),1)
      
      #contexutal loss
      loss_G_contextual = torch.mean(L1_criterion(X,X_hat),1)
      #entire loss in generator

      #encoder loss
      loss_G_encoder = torch.mean(L1_criterion(z,z_hat),1)
      
      loss_G = (loss_G_latent + loss_G_contextual + loss_G_encoder)/3
      
      loss_G_normal = (loss_G[y_aclabel == 1])
      loss_G_anomaly = pow(loss_G[y_aclabel == -1],-1)
      
      if loss_G_anomaly.size(0)>0:
        loss_G = (1-eta)*(sum(loss_G_normal)/loss_G_normal.size(0))+\
                eta*(sum(loss_G_anomaly)/loss_G_anomaly.size(0))
      else:
        loss_G = sum(loss_G_normal)/loss_G_normal.size(0)
      
      loss_G.backward()
      optimizer_G.step()

      if (i%50 == 0) & print_loss:
        print('[%d/%d] [%d/%d] Loss D: %.4f / Loss G: %.4f' % (epoch+1,epochs,i,len(dataloader),loss_D,loss_G))

      loss_D_list.append(loss_D)
      loss_G_list.append(loss_G)

In [9]:
def evaluation(data_tensor,model):
  if is_cuda:
    data_tensor = data_tensor.cuda()
    
  z,_,z_hat = model(data_tensor)

  L1_criterion = nn.L1Loss(reduction='none')
  score = L1_criterion(z,z_hat)
  score = torch.sum(score,dim=1).cpu().detach().numpy()
    
  return score

Global Parameters

In [10]:
seed_pool = [1,2,3,4,5]
anomaly_ratio_pool = [0.03,0.04,0.05]
#contam_ratio_pool = [0.98,0.5,0.0]
contam_ratio_pool = [0.5]
#hybrid hyper-parameter
eta = 0.5
#random search size
search_size = 10

In [11]:
epochs = 20
batch_size = 32

hyper_act_fun = [nn.Tanh(),nn.LeakyReLU()]
hyper_lr = [1e-2,1e-3,1e-4]
hyper_mom = [0.5,0.7,0.9]

hyper_list_entire = list(product(hyper_act_fun,hyper_lr,hyper_mom))

In [12]:
def random_search(hyper_list_entire,search_size,seed):
  if search_size < len(hyper_list_entire):
    set_seed(seed)
    index = np.random.choice(np.arange(len(hyper_list_entire)),search_size,replace = False)

    hyper_list = []
    for i in index:
      hyper_list.append(hyper_list_entire[i])
  else:
    hyper_list = hyper_list_entire

  return hyper_list

In [13]:
for contam_ratio in tqdm(contam_ratio_pool):
  df_result = pd.DataFrame(data = None,index = ['AUCPR'] + anomaly_ratio_pool,columns = seed_pool) 

  for seed in tqdm(seed_pool):
    #############################################seleting the best hyper-parameters in validation set#############################################
    metric_value_list = list()
    hyper_list = random_search(hyper_list_entire,search_size,seed)
    for i in range(len(hyper_list)):
      try:
        print(f'Finding Optimal Hyper-parameters......Current Candidates: {hyper_list[i]}')
        act_fun,lr,mom = hyper_list[i]
        #data
        train_loader,X_val_tensor,y_val,_,_ = data_prepare(batch_size,contam_ratio,hybrid = True,seed = seed)
        #model initialization
        set_seed(seed)
        net_generator=generator(input_size = X_val_tensor.size(1),act_fun = act_fun)
        net_discriminator=discriminator(input_size = X_val_tensor.size(1),act_fun = act_fun)

        if is_cuda:
          net_generator.cuda()
          net_discriminator.cuda()

        optimizer_G = torch.optim.SGD(net_generator.parameters(),lr = lr,momentum = mom)
        optimizer_D = torch.optim.SGD(net_discriminator.parameters(),lr = lr,momentum = mom)
        #fitting
        fit(dataloader = train_loader,net_generator = net_generator,net_discriminator = net_discriminator,
          eta = eta,epochs = epochs,batch_size = batch_size,print_loss = False)
        
        #evaluation
        score = evaluation(data_tensor = X_val_tensor,model = net_generator)
        metric_value = average_precision_score(y_true = y_val,y_score = score,pos_label = -1)
        metric_value_list.append(metric_value)

        print(f'The metric value corresponded to the hyper-parameters is :{metric_value:{.4}}')
        print('******************************')
        print('\n')
      except:
        #keep the right index
        metric_value_list.append(0)
        pass
      continue

    best_hyper_params=hyper_list[metric_value_list.index(max(metric_value_list))]
    print(f'The best hyper-parameters are: {best_hyper_params}')
    print('\n')
    ###################################################################testing#########################################################################
    print('Testing Phrase......')
    act_fun,lr,mom = best_hyper_params

    #data
    train_loader,_,_,X_test_tensor,y_test = data_prepare(batch_size,contam_ratio,hybrid = True,seed = seed)
    
    #model initialization, there exists randomness because of weight initialization***
    set_seed(seed)
    net_generator=generator(input_size = X_test_tensor.size(1),act_fun = act_fun)
    net_discriminator=discriminator(input_size = X_test_tensor.size(1),act_fun = act_fun)

    if is_cuda:
      net_generator.cuda()
      net_discriminator.cuda()

    optimizer_G = torch.optim.SGD(net_generator.parameters(),lr = lr,momentum = mom)
    optimizer_D = torch.optim.SGD(net_discriminator.parameters(),lr = lr,momentum = mom)
    #fitting
    fit(dataloader = train_loader,net_generator = net_generator,net_discriminator = net_discriminator,
      eta = eta,epochs = epochs,batch_size = batch_size,print_loss = False)
    #evaluation
    score = evaluation(data_tensor = X_test_tensor,model = net_generator)
    
    #store the result
    #AUCPR
    df_result.loc['AUCPR',seed] = average_precision_score(y_true = y_test,y_score = score,pos_label = -1)
    #F1
    for anomaly_ratio in anomaly_ratio_pool:
        threshold = score[np.argsort(-score)][int(anomaly_ratio*len(score))]
        
        y_pred = np.ones(len(score))
        y_pred[score >= threshold] = -1
        
        print('\n')
        print(f'Precision: {round(precision_score(y_pred = y_pred, y_true = y_test, pos_label= -1)*100,2)}')
        print(f'Recall: {round(recall_score(y_pred = y_pred, y_true = y_test, pos_label= -1)*100,2)}')
        print(f'F1-score: {round(f1_score(y_pred = y_pred, y_true = y_test, pos_label= -1)*100,2)}')
        print('\n')

        df_result.loc[anomaly_ratio,seed] = f1_score(y_pred = y_pred,y_true = y_test,pos_label = -1) 
  
  #mean & sd
  df_result['mean'] = np.mean(df_result.loc[:,seed_pool],axis = 1)
  df_result['std'] = np.std(df_result.loc[:,seed_pool],axis = 1)
  df_result = round(df_result.astype('float64')*100,2)

  file_path = 'E:/Research_Anomaly Detection/Taiwanese Bankruptcy Prediction (TBP)/result/' + 'TBP_HGAN_' + str(contam_ratio) + '.csv'
  df_result.to_csv(file_path,index = False)

  0%|                                                                                            | 0/5 [00:00<?, ?it/s]

Finding Optimal Hyper-parameters......Current Candidates: (Tanh(), 0.0001, 0.5)
The left true(known) positive samples in the training set:85/170

The metric value corresponded to the hyper-parameters is :0.008167
******************************


Finding Optimal Hyper-parameters......Current Candidates: (Tanh(), 0.001, 0.5)
The left true(known) positive samples in the training set:85/170

The metric value corresponded to the hyper-parameters is :0.08348
******************************


Finding Optimal Hyper-parameters......Current Candidates: (LeakyReLU(negative_slope=0.01), 0.001, 0.7)
The left true(known) positive samples in the training set:85/170

The metric value corresponded to the hyper-parameters is :0.06077
******************************


Finding Optimal Hyper-parameters......Current Candidates: (Tanh(), 0.01, 0.9)
The left true(known) positive samples in the training set:85/170

The metric value corresponded to the hyper-parameters is :0.1328
******************************





 20%|████████████████▌                                                                  | 1/5 [01:53<07:33, 113.38s/it]



Precision: 35.48
Recall: 68.75
F1-score: 46.81




Precision: 29.27
Recall: 75.0
F1-score: 42.11




Precision: 23.3
Recall: 75.0
F1-score: 35.56


Finding Optimal Hyper-parameters......Current Candidates: (LeakyReLU(negative_slope=0.01), 0.01, 0.5)
The left true(known) positive samples in the training set:85/170

The metric value corresponded to the hyper-parameters is :0.1277
******************************


Finding Optimal Hyper-parameters......Current Candidates: (Tanh(), 0.001, 0.7)
The left true(known) positive samples in the training set:85/170

The metric value corresponded to the hyper-parameters is :0.09742
******************************


Finding Optimal Hyper-parameters......Current Candidates: (LeakyReLU(negative_slope=0.01), 0.0001, 0.7)
The left true(known) positive samples in the training set:85/170

The metric value corresponded to the hyper-parameters is :0.009683
******************************


Finding Optimal Hyper-parameters......Current Candidates: (Tanh(), 0.0


 40%|█████████████████████████████████▏                                                 | 2/5 [03:49<05:42, 114.29s/it]

...



Precision: 37.1
Recall: 71.88
F1-score: 48.94




Precision: 31.71
Recall: 81.25
F1-score: 45.61




Precision: 27.18
Recall: 87.5
F1-score: 41.48


Finding Optimal Hyper-parameters......Current Candidates: (Tanh(), 0.0001, 0.5)
The left true(known) positive samples in the training set:85/170

The metric value corresponded to the hyper-parameters is :0.007807
******************************


Finding Optimal Hyper-parameters......Current Candidates: (Tanh(), 0.001, 0.5)
The left true(known) positive samples in the training set:85/170

The metric value corresponded to the hyper-parameters is :0.03693
******************************


Finding Optimal Hyper-parameters......Current Candidates: (LeakyReLU(negative_slope=0.01), 0.0001, 0.7)
The left true(known) positive samples in the training set:85/170

The metric value corresponded to the hyper-parameters is :0.008249
******************************


Finding Optimal Hyper-parameters......Current Candidates: (LeakyReLU(negative_slope=0.01)


 80%|██████████████████████████████████████████████████████████████████▍                | 4/5 [07:43<01:55, 115.59s/it]



Precision: 35.48
Recall: 68.75
F1-score: 46.81




Precision: 31.71
Recall: 81.25
F1-score: 45.61




Precision: 26.21
Recall: 84.38
F1-score: 40.0


Finding Optimal Hyper-parameters......Current Candidates: (Tanh(), 0.001, 0.9)
The left true(known) positive samples in the training set:85/170

The metric value corresponded to the hyper-parameters is :0.1053
******************************


Finding Optimal Hyper-parameters......Current Candidates: (Tanh(), 0.01, 0.7)
The left true(known) positive samples in the training set:85/170

The metric value corresponded to the hyper-parameters is :0.1124
******************************


Finding Optimal Hyper-parameters......Current Candidates: (LeakyReLU(negative_slope=0.01), 0.01, 0.7)
The left true(known) positive samples in the training set:85/170

The metric value corresponded to the hyper-parameters is :0.1348
******************************


Finding Optimal Hyper-parameters......Current Candidates: (Tanh(), 0.01, 0.9)
The left true(known


100%|███████████████████████████████████████████████████████████████████████████████████| 1/1 [09:40<00:00, 580.08s/it]



Precision: 35.48
Recall: 68.75
F1-score: 46.81




Precision: 30.49
Recall: 78.12
F1-score: 43.86




Precision: 25.24
Recall: 81.25
F1-score: 38.52


